In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [10]:
df = pd.read_csv("states_all.csv")

In [15]:
 for i in np.arange(3,len(df.columns)):
    df[df.columns[i]].interpolate(method ='linear', limit_direction ='forward',inplace = True)

In [16]:
for i in np.arange(3,len(df.columns)):
    df[df.columns[i]] = df[df.columns[i]].fillna(df.groupby('YEAR')[df.columns[i]].transform(lambda x: x.fillna(x.mean())))

In [17]:
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,798454.929179,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,555625.588825,115705.842048,731634.0,208.000000,252.000000,207.000000,260.0
1,1992_ALASKA,ALASKA,1992,798454.929179,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,555625.588825,115705.842048,122487.0,218.418605,266.325581,215.372093,260.0
2,1992_ARIZONA,ARIZONA,1992,798454.929179,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,555625.588825,115705.842048,673477.0,215.000000,265.000000,209.000000,260.0
3,1992_ARKANSAS,ARKANSAS,1992,798454.929179,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,555625.588825,115705.842048,441490.0,210.000000,256.000000,211.000000,260.0
4,1992_CALIFORNIA,CALIFORNIA,1992,798454.929179,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,555625.588825,115705.842048,5254844.0,208.000000,261.000000,202.000000,260.0


**(1)** Create a variable that contains the weighted average of the grades in the dataset. The number of students in the fourth grade is different from that of the eighth grade. So you will need a weighted average!

In [23]:
df["weighted average"] = (df["GRADES_8_G"]*(df["AVG_MATH_8_SCORE"]+df["AVG_READING_8_SCORE"])+ df["GRADES_4_G"]*(df["AVG_MATH_4_SCORE"]+df["AVG_READING_4_SCORE"]))/(df["GRADES_4_G"]+df["GRADES_8_G"])

In [78]:
df["weighted average"]

0       463.532201
1       477.664523
2       471.430784
3       469.427233
4       461.586457
           ...    
1710    509.554393
1711    505.474412
1712    485.520116
1713    508.462987
1714    511.554393
Name: weighted average, Length: 1715, dtype: float64

**(2)** What is the correlation between the variable you just created and the types of expenditures? Which expenditure item has more correlation than others?

In [35]:
df.corr()["weighted average"].head(11).sort_values(ascending=False)

YEAR                            0.488562
LOCAL_REVENUE                   0.167914
SUPPORT_SERVICES_EXPENDITURE    0.165798
TOTAL_REVENUE                   0.155918
INSTRUCTION_EXPENDITURE         0.153773
TOTAL_EXPENDITURE               0.150782
FEDERAL_REVENUE                 0.150214
STATE_REVENUE                   0.133869
OTHER_EXPENDITURE               0.125356
CAPITAL_OUTLAY_EXPENDITURE      0.078447
ENROLL                          0.024720
Name: weighted average, dtype: float64

**local revenue has highest correlation with weighted average**

In [53]:
df.iloc[:,8:12]

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE
0,2.653798e+06,1.481703e+06,7.350360e+05,262049.247156
1,9.724880e+05,4.983620e+05,3.509020e+05,262049.247156
2,3.401580e+06,1.435908e+06,1.007732e+06,262049.247156
3,1.743022e+06,9.643230e+05,4.834880e+05,262049.247156
4,2.713883e+07,1.435892e+07,8.520926e+06,262049.247156
...,...,...,...,...
1710,1.276276e+07,6.750863e+06,3.811953e+06,568480.588235
1711,1.276276e+07,6.750863e+06,3.811953e+06,568480.588235
1712,1.276276e+07,6.750863e+06,3.811953e+06,568480.588235
1713,1.276276e+07,6.750863e+06,3.811953e+06,568480.588235


**(3)** Now apply the Principal Components Analysis (PCA) for the four expenditure items! How much of the total variance can be explained by the first component?

In [54]:
df_expend = df.iloc[:,8:12]

In [57]:
X = StandardScaler().fit_transform(df_expend)
pca = PCA(n_components=4)
principalComponents=pca.fit_transform(X)

In [60]:
exp_var= pca.explained_variance_ratio_
cumsum_var=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
cumsum_var[0]

97.17

**(4)** What is the correlation between the GPA you created and the first principal component?

In [70]:
df["pc1"] = principalComponents[:,0]

In [77]:
df["weighted average"].corr(df["pc1"])

0.1512379862969346

**(5)** When you need to choose the most appropriate variables for your model, would you prefer the first basic variables instead of the expenditure items? Why?

**yes instade of four different element i can take just one feature with PCA. it will reduce the complexity of our model. i can explain the 97 percent of these expenditures with just one variable.**
